In [1]:
import numpy as np
import cv2
import random
import math
import copy

In [2]:
def remove_padding(image,size):
    len1=len(image)
    len2=len(image[0])
    size=int(size/2)
    for i in range(size):
        del image[0]
        
    for i in range(size):
        del image[len1-2*size]

    for i in range(len(image)):
        for j in range(size):
            del image[i][0]
        
        for j in range(size):
            del image[i][len2-2*size]
    return image
    


In [3]:
def padding(image,size):
    size=int(size/2)
    len1,len2=image.shape
    pad=[]
    for i in range(0,(len1+2*size)):
        temp=[]
        for j in range(0,(len2+2*size)):
            if i<size or j<size  or j>=(len2+size) or i>=(len1+size):
                temp.append(0)
            else:
                temp.append(image[i-size][j-size])
        pad.append(temp)
    return pad


In [4]:
def gaussian_func(x,y,std):
    value=(-1)*(x**2+y**2)/float(2*(std**2))
    return math.exp(value)/float((2*math.pi*(std**2)))



In [5]:
def summing(image):
    length1=len(image)
    length2=len(image[0])
    counter=0
    for i in range(length1):
        for j in range(length2):
            counter=counter+image[i][j]
    return counter
    

In [6]:
def gaussian_filter(image,kernel_func,i,j):
    len1=len(kernel_func)
    len2=len1
    len1=int(len1/2)
    filter_space=[]
    for k in range((i-len1),(i+len1+1)):
        temp=[]
        for l in range((j-len1),(j+len1+1)):
            temp.append(image[k][l])
        filter_space.append(temp)
    
    final_array=(np.array(filter_space).reshape(len2,len2))*(np.array(kernel_func).reshape(len2,len2))
    return summing(final_array)

In [7]:
def gaussian_blur(kernel,image_main):
    kernel_space=[]
    for i in range(kernel):
        temp=[]
        for j in range(kernel):
            temp.append(gaussian_func((i-1),(j-1),1))
        kernel_space.append(temp)
    kernel_space/=np.sum(list(kernel_space))
    pad1=[]
    #gaussian blur of size 3
    pad1=padding(image_main,5)
    pad2=copy.deepcopy(pad1)
    for i in range(2,(len(pad1)-2)):
        for j in range(2,(len(pad1[i])-2)):
            pad1[i][j]=gaussian_filter(pad2,list(kernel_space),i,j)
    pad1=remove_padding(pad1,5)
    b=np.array(pad1)
    return b

In [8]:
def downgrade(image):
    len1=len(image)
    len2=len(image[0])
    image_ret=[]
    for i in range(int(len1/2)):
        temp=[]
        for j in range(int(len2/2)):
            temp.append(image[i*2][j*2])
        image_ret.append(temp)
    return image_ret

In [9]:
def upgrade(image):
    len1=len(image)
    len2=len(image[0])
    image_ret=[]
    for i in range(int(len1*2)):
        temp=[]
        for j in range(int(len2*2)):
            temp.append(image[int(i/2)][int(j/2)])
        image_ret.append(temp)
    return image_ret

In [10]:
size_local=255

In [11]:
# Load an color image 
img = cv2.imread('image_3.png',0)
cv2.imshow('Main Image ',img)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [12]:
image_main1=copy.deepcopy(img[:])
image_main=copy.deepcopy(image_main1)
image_mai2=copy.deepcopy(image_main1)

In [13]:
gaussian_pyramid=[]
gaussian_pyramid.append(image_main)
for i in range(3):
    
    #5*5 gaussian filter 
    b=np.array
    b=gaussian_blur(5,image_main)
    cv2.imwrite('level'+str(i+1)+'_gaussianblur.png',(b/np.max(b)*size_local))
    cv2.imshow('Gaussian Filter 5*5  Gaussian Blur  ',b/np.max(b))
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    #Level 1 downgrade
    down1=downgrade(b)
    gaussian_pyramid.append(down1)
    cv2.imwrite('level'+str(i+1)+'_downgrade.png',(down1/np.max(down1)*size_local))
    cv2.imshow('Gaussian Filter 5*5  Gaussian Blur Downgrade'+str(i+1) ,down1/np.max(down1))
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    image_main=np.array(down1)

In [14]:
laplacian_pyramid=[]
laplacian_pyramid.append(gaussian_pyramid[3])
for i in range(3):

    b=upgrade(gaussian_pyramid[3-i])
    cv2.imwrite('level'+str(i+1)+'laplacian_expand.png',(b/np.max(b)*size_local))
    cv2.imshow('Laplacian Expand '+str(i+1) ,b/np.max(b))
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    result=np.subtract(gaussian_pyramid[3-i-1],b)
    result*=np.max(result)
#     result/=result/np.max(result)
    laplacian_pyramid.append(result)
    image_main1=np.array(gaussian_pyramid[3-i-1])
    cv2.imwrite('level'+str(i+1)+'Subtraction.png',(result/np.max(result)*size_local))
    cv2.imshow('Laplacian Formation by Subtraction'+str(i+1) ,result/np.max(result))
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    
    